In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm
from Model import *

In [2]:
tf.random.set_seed(7)

In [3]:
train = pd.read_csv("../../dataset/goodreads_train.csv")
vocabulary2 = np.load('../../vocabulaires/voc_without_std_word_count_5.npy', allow_pickle=True)

In [4]:
#model_list = [cnn1,cnn2, cnn3,cnn4,cnn5,cnn6, cnn8, cnn9, cnn10]
model_list = [cnn8]
for model_obj in model_list:
    model = model_obj.Model(vocabulary2)
    print(model.name)
    if not os.path.exists(f"logs/{model.name}"):
        os.mkdir(f"logs/{model.name}")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=f"logs/{model.name}")
    model.run_experiment([train['review_text'], train['n_comments'], train['n_votes']], train['rating'], epochs=50, tensorboard_callback=tensorboard_callback, batch_size=1500)

cnn8_dropout
Epoch 1/50
480/480 [==============================] - 89s 177ms/step - loss: 2.0499 - categorical_accuracy: 0.1550 - val_loss: 1.8706 - val_categorical_accuracy: 0.1398
Epoch 2/50
480/480 [==============================] - 83s 173ms/step - loss: 1.8041 - categorical_accuracy: 0.1599 - val_loss: 1.7362 - val_categorical_accuracy: 0.2019
Epoch 3/50
480/480 [==============================] - 83s 172ms/step - loss: 1.7012 - categorical_accuracy: 0.2377 - val_loss: 1.6674 - val_categorical_accuracy: 0.2592
Epoch 4/50
480/480 [==============================] - 83s 173ms/step - loss: 1.5878 - categorical_accuracy: 0.3028 - val_loss: 1.5055 - val_categorical_accuracy: 0.3228
Epoch 5/50
480/480 [==============================] - 83s 174ms/step - loss: 1.5066 - categorical_accuracy: 0.3446 - val_loss: 1.4141 - val_categorical_accuracy: 0.3900
Epoch 6/50
480/480 [==============================] - 83s 173ms/step - loss: 1.4563 - categorical_accuracy: 0.3939 - val_loss: 1.3276 - val_ca